# Compute Notebook Data

This notebook is devoted to calculating notebok statistics for further analysis. There are generally three types of stats we aim to calculate:
1. Notebook metadata stored in the notebook itself (e.g., nbformat, kernel_lang)
2. Descriptive information about cells, their type, and length
3. Descriptive information about the content of cells (import statements, keywords, links)

In [1]:
import os
import re
import time
import json
import datetime

import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
%load_ext line_profiler

## 0. Load notebook, repo, and readme metadata

In [12]:
# Notebook metadata
df_nb = pd.read_csv('../data/csv/nb_metadata.csv')
df_nb.rename(columns = {'Unnamed: 0':'nb_id'}, inplace = True)
print(df_nb.shape)
df_nb.head()

(1253620, 16)


,nb_id,html_url,max_filesize,min_filesize,name,owner_html_url,owner_id,owner_login,path,query_page,repo_description,repo_fork,repo_html_url,repo_id,repo_name,repo_private
0,0,https://github.com/dalequark/emotivExperiment/...,10,0,EmotivDataAnalysis.ipynb,https://github.com/dalequark,2328571,dalequark,ipynb/EmotivDataAnalysis.ipynb,1,NaN,False,https://github.com/dalequark/emotivExperiment,26093748,emotivExperiment,False
1,1,https://github.com/kevcisme/madelon_redux/blob...,10,0,Part_IV_Project_3-checkpoint_BASE_63907.ipynb,https://github.com/kevcisme,24496260,kevcisme,ipynb/.ipynb_checkpoints/Part_IV_Project_3-che...,1,NaN,False,https://github.com/kevcisme/madelon_redux,95729593,madelon_redux,False
2,2,https://github.com/HaraldoFilho/DLND-Projects/...,10,0,_.ipynb,https://github.com/HaraldoFilho,15271881,HaraldoFilho,_.ipynb,1,"Index for the projects of the Udacity's ""Deep ...",False,https://github.com/HaraldoFilho/DLND-Projects,88182909,DLND-Projects,False
3,3,https://github.com/mhjensen/CPMLS/blob/4a5b37e...,10,0,csexmas2015.ipynb,https://github.com/mhjensen,2732953,mhjensen,doc/pub/CSETalks/csexmas2015/ipynb/csexmas2015...,1,Master program in Computational Science. The l...,False,https://github.com/mhjensen/CPMLS,35169104,CPMLS,False
4,4,https://github.com/freqn/atom_configuration/bl...,10,0,jupyter.ipynb,https://github.com/freqn,3611075,freqn,packages/file-icons/examples/jupyter.ipynb,1,Atom Config,False,https://github.com/freqn/atom_configuration,57460377,atom_configuration,False


In [3]:
# Repository metadata
df_repo = pd.read_csv('../data/csv/repo_metadata.csv')
del df_repo['Unnamed: 0']
print(df_repo.shape)
df_repo.head()

(193026, 24)


,created_at,description,fork,forks_count,has_downloads,has_issues,has_pages,has_wiki,html_url,id,...,owner_id,owner_login,owner_type,private,pushed_at,size,stargazers_count,subscribers_count,updated_at,watchers_count
0,2008-03-01T23:35:48Z,Pysolar is a collection of Python libraries fo...,False,59,True,True,True,False,https://github.com/pingswept/pysolar,2058,...,1875,pingswept,User,False,2017-05-29T13:34:12Z,4400,114,24,2017-07-10T09:46:44Z,114
1,2008-10-22T10:30:25Z,My uni homework,False,0,True,True,False,True,https://github.com/ELLIOTTCABLE/Homework,66233,...,200,ELLIOTTCABLE,User,False,2016-10-05T10:48:54Z,634,4,3,2016-05-08T15:22:05Z,4
2,2008-12-09T21:05:22Z,"Incubator for useful bioinformatics code, prim...",False,172,True,True,True,False,https://github.com/chapmanb/bcbb,87831,...,39391,chapmanb,User,False,2017-07-23T10:26:58Z,67640,368,54,2017-07-25T13:10:17Z,368
3,2009-02-01T23:43:19Z,NaN,False,2,True,True,False,True,https://github.com/certik/chess,119491,...,20568,certik,User,False,2016-03-20T22:53:21Z,3006,4,4,2016-05-09T00:40:15Z,4
4,2009-02-02T22:26:08Z,Generic Disease Modelling System,False,5,True,True,False,True,https://github.com/aflaxman/gbd,120233,...,51236,aflaxman,User,False,2013-02-27T22:06:57Z,69051,11,3,2016-08-30T23:29:41Z,11


In [4]:
# Readme metadtata
df_readme = pd.read_csv('../data/csv/repo_readme.csv')
del df_readme['Unnamed: 0']
print(df_readme.shape)
df_readme.head()

(142449, 7)


,content,html_url,name,path,repo_id,size,type
0,IyMgUHlzb2xhciAjIwoKWyFbQnVpbGQgU3RhdHVzXShodH...,https://github.com/pingswept/pysolar/blob/mast...,README.markdown,README.markdown,2058,3226,file
1,SG9tZXdvcmsKPT09PT09PT0KV2hhdCB0aGUgdGl0bGUgc2...,https://github.com/ELLIOTTCABLE/Homework/blob/...,README.markdown,README.markdown,66233,441,file
2,Q29sbGVjdGlvbiBvZiB1c2VmdWwgY29kZSByZWxhdGVkIH...,https://github.com/chapmanb/bcbb/blob/master/R...,README.md,README.md,87831,1171,file
3,SW50cm9kdWN0aW9uDQo9PT09PT09PT09PT0NCg0KVGhpcy...,https://github.com/aflaxman/gbd/blob/master/RE...,README.rst,README.rst,120233,690,file
4,Q0FQTyBOb3RlczoKUGFzdGVkIGZyb20gUEFQRVIgd2lraS...,https://github.com/HERA-Team/capo/blob/master/...,README,README,151115,1086,file


## 1. Scrape Additional nb data from nb file

We currently only have the metadata provided from Github's API. Now we can look at the nb files themselves and get the metadata from within the notebook (nbformat, kernel, langauge) and compute stats about each notebook (type and num of cell, and so on).

For now let's test this metadata extraction with a subset of 1000 nbs. And if that works we can then apply the extraction code to all notebooks.

In [18]:
df_nb_sample = df_nb.sample(1000)
df_nb_sample.shape

(1000, 23)

In [5]:
def write_to_log(msg):
    f = '../logs/nb_parse_log.txt'
    log_file = open(f, "a")
    log_file.write(msg + "\n")
    
def get_nb_metadata(df):
    # create blank columns for the data
    df["nbformat"] = None
    df["nbformat_minor"] = None
    df["kernel_lang"] = None
    df["kernel_name"] = None
    df["lang_name"] = None
    df["lang_version"] = None
    df['num_cells'] = None
    
    # track time to process files
    count = 0    
    time1 = time.time()
    
    for index, row in df.iterrows():
        # track time to process files
        count += 1
        if count % 10000 == 0:
            print('%s rows in %s seconds' % (count, time.time() - time1))

        f = '../data/notebooks/nb_%s.ipynb' % row['nb_id']
        with open(f) as data_file:
            date_string = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            try:
                data = json.load(data_file)
            except:
                # track files we could not open
                msg = '%s: had trouble loading %s' % (date_string, row['nb_id'])
                write_to_log(msg)
                continue

            if isinstance(data, dict): 
                keys = data.keys()
            else:
                keys = []

            # get nb top level metadata
            if 'nbformat' in keys:
                df.set_value(index, 'nbformat', data['nbformat'])
            if 'nbformat_minor' in keys:
                df.set_value(index, 'nbformat_minor', data['nbformat_minor'])
            
            # get information about the number of cells
            if 'cells' in keys:
                df.set_value(index, 'num_cells', len(data['cells']))
            elif 'worksheets' in keys:
                num_cells = 0
                for w in data['worksheets']:
                    num_cells += len(w['cells'])
                df.set_value(index, 'num_cells', num_cells)

            # get info from the metadata dictionary
            if 'metadata' in keys:
                metadata_keys = data['metadata'].keys()

                if 'kernelspec' in metadata_keys:
                    kernel_keys = data['metadata']['kernelspec'].keys()
                    if 'language' in kernel_keys:
                        df.set_value(index, 'kernel_lang', data['metadata']['kernelspec']['language'])
                    if 'display_name' in kernel_keys:
                        df.set_value(index, 'kernel_name', data['metadata']['kernelspec']['display_name'])

                if 'language_info' in metadata_keys:
                    lang_keys = data['metadata']['language_info'].keys()
                    if 'name' in lang_keys:
                        df.set_value(index, 'lang_name', data['metadata']['language_info']['name'])
                    if 'version' in lang_keys:
                        df.set_value(index, 'lang_version', data['metadata']['language_info']['version'])

                elif 'language' in metadata_keys:
                    df.set_value(index, 'lang_name', data['metadata']['language'])

We eventually got the code to work on the 1000 sample notebooks, so now we will apply it to the whole dataset. Here, as elsewhere I track how long the extraction is taking.

In [6]:
# get metadata and save updated dataframe to a csv file
get_nb_metadata(df_nb)
df_nb.to_csv('../data/csv/nb_metadata_w_nb.csv')

10000 rows in 66.51938724517822 seconds
20000 rows in 106.40936398506165 seconds
30000 rows in 233.25157499313354 seconds
40000 rows in 280.3122229576111 seconds
50000 rows in 327.68263602256775 seconds
60000 rows in 431.2840702533722 seconds
70000 rows in 475.36883091926575 seconds
80000 rows in 515.8759350776672 seconds
90000 rows in 693.012088060379 seconds
100000 rows in 741.2404401302338 seconds
110000 rows in 783.0808730125427 seconds
120000 rows in 824.5476369857788 seconds
130000 rows in 869.077358007431 seconds
140000 rows in 912.9306890964508 seconds
150000 rows in 1070.3463661670685 seconds
160000 rows in 1116.3673310279846 seconds
170000 rows in 1156.7598099708557 seconds
180000 rows in 1214.2917590141296 seconds
190000 rows in 1258.8786220550537 seconds
200000 rows in 1502.3247511386871 seconds
210000 rows in 1548.7590849399567 seconds
220000 rows in 1600.6675288677216 seconds
230000 rows in 1653.0572032928467 seconds
240000 rows in 1697.7411050796509 seconds
250000 rows i

## 2. Get Data about Cell Types

I want to track data on each cell. I think the best way to do this is to have a dataframe with a separate row for each cell. I think I want to track the following data. Some of the difficulty in writing this extraction code will be handling nbformat 3.0 that use a different format to store notebook data.

1. nb_id
2. workbook_index
2. cell_index
3. cell_type
3. num_words (raw, markdown and headings)
4. lines of code (code cells)
5. num_execute_result
6. execute_result_keys
7. num_error
8. error_names
9. error_values
10. num_stream
11. num_display_data
12. display_data_keys

In [46]:
def write_to_log(msg):
    f = '../logs/cell_parse_log.txt'
    log_file = open(f, "a")
    log_file.write(msg + "\n")
    #log_file.close()   
    
def get_all_cell_data(df, tracking_trigger = 10000):
    
    all_cells = []      
    count = 0    
    time1 = time.time()
    
    for index, row in df.iterrows():
        
        # track progress through notebooks
        count += 1
        if count % tracking_trigger == 0:
            print('%s nbs in %s seconds' % (count, time.time() - time1))

        f = '../data/notebooks/nb_%s.ipynb' % row['nb_id']
        with open(f) as nb_file:
            date_string = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            try:
                data = json.load(nb_file)
            except:
                msg = '%s: had trouble loading nb %s' % (date_string, row['nb_id'])
                write_to_log(msg)
                continue

            if isinstance(data, dict): 
                keys = data.keys()
            else:
                keys = []

            # get data for each cell, v 4.0
            if 'cells' in keys:
                for i, c in enumerate(data['cells']):
                    cell_data = get_cell_data(c, i, row['nb_id'])
                    all_cells.append(cell_data)
            
            # get data for each cell, v 3.0
            elif 'worksheets' in keys:
                for j, w in enumerate(data['worksheets']):
                    
                    if isinstance(w, dict): 
                        worksheet_keys = w.keys()
                    else:
                        keys = []
                    
                    if 'cells' in worksheet_keys:
                        for k, c in enumerate(w['cells']):
                            cell_data = get_cell_data(c, k, row['nb_id'], j)
                            all_cells.append(cell_data)
                
    return all_cells
    
def get_cell_data(cell, index, nb_id, worksheet_index = None):
    nbformat_3_mimes = ['text', 'latex', 'png', 'jpeg', 'svg', 'html', 'javascript', 'json', 'pdf', 'metadata']
    
    if isinstance(cell, dict): 
        cell_keys = cell.keys()
    else:
        cell_keys = [] 
    
    if 'cell_type' in cell_keys:
        cell_type = cell['cell_type']
    else:
        cell_type = None
    
    
    if cell_type in ['raw','markdown','heading']:
        num_words = 0
        if 'source' in cell_keys:
            if isinstance(cell['source'], list):
                for l in cell['source']:
                    words = len(l.split())
                    num_words += words
            elif isinstance(cell['source'], str):
                num_words += len(cell['source'].split())
            else:
                num_words = None
    else:
        num_words = None
    
    if cell_type == 'code':
        lines_of_code = 0
        if 'source' in cell_keys:
            if isinstance(cell['source'], list):
                lines_of_code = len(cell['source'])
            elif isinstance(cell['source'], str):
                lines_of_code = len(cell['source'].splitlines())
            else:
                lines_of_code = None
            
        elif 'input' in cell_keys:
            if isinstance(cell['input'], list):
                lines_of_code = len(cell['input'])
            elif isinstance(cell['input'], str):
                lines_of_code = len(cell['input'].splitlines())
            else:
                lines_of_code = None
    else:
        lines_of_code = None
    
    
    # inilize output counts
    num_execute_result = 0
    num_error = 0
    num_stream = 0
    num_display_data = 0
    
    execute_result_keys = []
    error_names = []
    error_values = []
    display_data_keys = []
        
    if 'outputs' in cell_keys:
        for o in cell['outputs']:

            if isinstance(o, dict):
                output_keys = o.keys()
            else:
                output_keys = []

            if 'output_type' in output_keys:
                if o['output_type'] in ['execute_result','pyout']:
                    num_execute_result += 1
                    if 'data' in output_keys:
                        if isinstance(o['data'], dict):
                            data_keys = o['data'].keys()
                            for k in data_keys:
                                execute_result_keys.append(k)
                    else:
                        for k in output_keys:
                            if k in nbformat_3_mimes:
                                execute_result_keys.append(k)

                elif o['output_type'] in ['error','pyerr']:
                    num_error += 1
                    if 'ename' in output_keys:
                        error_names.append(o['ename'])
                    if 'evalue' in output_keys:
                        error_values.append(o['evalue'])

                elif o['output_type'] == 'stream':
                    num_stream += 1

                elif o['output_type'] == 'display_data':
                    num_display_data += 1
                    if 'data' in output_keys:
                        if isinstance(o['data'], dict):
                            data_keys = o['data'].keys()
                            for k in data_keys:
                                display_data_keys.append(k)
                    else:
                        for k in output_keys:
                            if k in nbformat_3_mimes:
                                display_data_keys.append(k)

    return [nb_id, worksheet_index, index, cell_type, num_words, lines_of_code, 
            num_execute_result, execute_result_keys, num_error, error_names, 
            error_values, num_stream, num_display_data, display_data_keys]


As is best practice, let's use a subset of the data to test our code

In [24]:
sample_cell_data = get_all_cell_data(df_nb_sample, 100)

100 nbs in 1.8204798698425293 seconds
200 nbs in 3.8096261024475098 seconds
300 nbs in 5.4645349979400635 seconds
400 nbs in 7.0867249965667725 seconds
500 nbs in 8.622582912445068 seconds
600 nbs in 9.964746952056885 seconds
700 nbs in 11.574764013290405 seconds
800 nbs in 13.0364248752594 seconds
900 nbs in 14.345754146575928 seconds
1000 nbs in 15.574757099151611 seconds


And let's see how the celld ata looks

In [28]:
df_cell = pd.DataFrame(sample_cell_data)
print(df_cell.shape)
df_cell.columns = ['nb_id','workbook_index','cell_index','cell_type','num_words','lines_of_code',
                   'num_execute_result','execute_result_keys','num_error','error_names','error_values',
                   'num_stream','num_display_data','display_data_keys']
df_cell.head(20)

(21560, 14)


,nb_id,workbook_index,cell_index,cell_type,num_words,lines_of_code,num_execute_result,execute_result_keys,num_error,error_names,error_values,num_stream,num_display_data,display_data_keys
0,950229,None,0,markdown,5.0,NaN,0,[],0,[],[],0,0,[]
1,950229,None,1,code,NaN,8.0,0,[],0,[],[],0,0,[]
2,950229,None,2,markdown,7.0,NaN,0,[],0,[],[],0,0,[]
3,950229,None,3,code,NaN,2.0,1,[text/plain],0,[],[],0,0,[]
4,950229,None,4,code,NaN,14.0,0,[],0,[],[],1,0,[]
5,950229,None,5,markdown,11.0,NaN,0,[],0,[],[],0,0,[]
6,950229,None,6,code,NaN,6.0,0,[],0,[],[],1,0,[]
7,950229,None,7,code,NaN,1.0,0,[],0,[],[],1,0,[]
8,950229,None,8,code,NaN,1.0,0,[],0,[],[],0,0,[]
9,950229,None,9,code,NaN,1.0,0,[],0,[],[],1,0,[]


In [31]:
df_cell.to_csv('../data/csv/cell_metadata_sample.csv')

Alright, that computing of cell data seemed to work on 1000 notebooks, now for the 1.25 million

In [36]:
all_cell_data = get_all_cell_data(df_nb, 10000)

df_cell = pd.DataFrame(all_cell_data)
print(df_cell.shape)
df_cell.columns = ['nb_id','workbook_index','cell_index','cell_type','num_words','lines_of_code',
                   'num_execute_result','execute_result_keys','num_error','error_names','error_values',
                   'num_stream','num_display_data','display_data_keys']

df_cell.to_csv('../data/csv/cell_metadata.csv')

10000 nbs in 51.6393301486969 seconds
20000 nbs in 96.67775797843933 seconds
30000 nbs in 211.67645001411438 seconds
40000 nbs in 261.78705406188965 seconds
50000 nbs in 318.10353207588196 seconds
60000 nbs in 427.5553991794586 seconds
70000 nbs in 474.3860511779785 seconds
80000 nbs in 516.8671221733093 seconds
90000 nbs in 701.8724331855774 seconds
100000 nbs in 750.3447201251984 seconds
110000 nbs in 798.1056818962097 seconds
120000 nbs in 839.1192629337311 seconds
130000 nbs in 884.4429590702057 seconds
140000 nbs in 940.4327092170715 seconds
150000 nbs in 1103.9290418624878 seconds
160000 nbs in 1145.1924579143524 seconds
170000 nbs in 1196.874055147171 seconds
180000 nbs in 1256.7895259857178 seconds
190000 nbs in 1299.6017332077026 seconds
200000 nbs in 1560.0286478996277 seconds
210000 nbs in 1612.138699054718 seconds
220000 nbs in 1665.2173948287964 seconds
230000 nbs in 1720.3766241073608 seconds
240000 nbs in 1770.952070236206 seconds
250000 nbs in 1930.45654296875 seconds
2

## Scrape nbformat 2.0 and 3.0 cells

Coming back from a break, it appears I did not check my code closely last time as I have a large number of files that did not get parsed. These are mainly the nbformat 2.0 and 3.0 files that store cells in a worksheet.

First up, I need a list of those files that did not make it into the list, igoring the ones that did not parse since these are simply due to malformatted json.

In [38]:
missing_cell_data = []

with open('../logs/cell_parse_log.txt', 'r') as f:
    for l in f:
        missing_cell_data.append(int(l.split(' ')[6]))

missing_cell_data = set(missing_cell_data)
missing_cell_data = np.array(list(missing_cell_data))
len(missing_cell_data)

12054

In [39]:
df_cell = pd.read_csv('../data/csv/cell_metadata.csv')
nbs_with_cell_data = df_cell.nb_id.unique()

In [41]:
nbs_not_reparse = np.unique(np.append(missing_cell_data, nbs_with_cell_data))
len(nbs_not_reparse)

1079894

In [42]:
df_nbs_to_reparse = df_nb[~df_nb.nb_id.isin(nbs_not_reparse)]
df_nbs_to_reparse.shape

(173726, 23)

Okay, so we have ~173k notebooks that were not parsed and that are not a part of our known problem notebooks (i.e. ones with malformatted json that we could not get metadata from in step 1)

Let's test the revised code to make sure it works.

In [47]:
df_nbs_to_reparse_sample = df_nbs_to_reparse.sample(1000)

sample_reparse = get_all_cell_data(df_nbs_to_reparse_sample, 100)

100 nbs in 1.71806001663208 seconds
200 nbs in 3.44852614402771 seconds
300 nbs in 5.113385915756226 seconds
400 nbs in 6.739964008331299 seconds
500 nbs in 8.426445007324219 seconds
600 nbs in 10.00645899772644 seconds
700 nbs in 11.894474029541016 seconds
800 nbs in 13.5018949508667 seconds
900 nbs in 15.192470073699951 seconds
1000 nbs in 16.959682941436768 seconds


In [48]:
df_sample_reparse = pd.DataFrame(sample_reparse)

In [53]:
df_sample_reparse.shape
df_sample_reparse.columns = ['nb_id','workbook_index','cell_index','cell_type','num_words','lines_of_code',
                   'num_execute_result','execute_result_keys','num_error','error_names','error_values',
                   'num_stream','num_display_data','display_data_keys']

In [55]:
len(df_sample_reparse['nb_id'].unique())

987

Yep, of the 1000 notebooks we tried to get data on, 13 didn't work. That is likely to more JSON malformatting issues. Okay, that sample worked, so now for all the missing data

In [56]:
reparse_cell_data = get_all_cell_data(df_nbs_to_reparse, 10000)

df_cell_reparse = pd.DataFrame(reparse_cell_data)
print(df_cell_reparse.shape)
df_cell_reparse.columns = ['nb_id','workbook_index','cell_index','cell_type','num_words','lines_of_code',
                   'num_execute_result','execute_result_keys','num_error','error_names','error_values',
                   'num_stream','num_display_data','display_data_keys']

df_cell_reparse.to_csv('../data/csv/cell_metadata_v2-3.csv')

10000 nbs in 96.36146402359009 seconds
20000 nbs in 194.91301703453064 seconds
30000 nbs in 448.3327031135559 seconds
40000 nbs in 548.8574161529541 seconds
50000 nbs in 681.528135061264 seconds
60000 nbs in 799.5319352149963 seconds
70000 nbs in 904.6161372661591 seconds
80000 nbs in 1007.1710021495819 seconds
90000 nbs in 1117.6736481189728 seconds
100000 nbs in 1200.3324780464172 seconds
110000 nbs in 1310.8296191692352 seconds
120000 nbs in 1388.7029349803925 seconds
130000 nbs in 1501.5205039978027 seconds
140000 nbs in 1571.8860640525818 seconds
150000 nbs in 1660.7107701301575 seconds
160000 nbs in 1728.6355922222137 seconds
170000 nbs in 1798.524698972702 seconds
(4693111, 14)


In [69]:
reparsed_nbs = df_cell_reparse.nb_id.unique()

In [71]:
print(len(reparsed_nbs) + len(nbs_with_cell_data))
len(np.intersect1d(reparsed_nbs, nbs_not_reparse, assume_unique=True))

1239355


0

Between our first pass (which mainly got data from nbformat 4.0 notebooks) and this pass (which included 2.0 and 3.0 notebooks) we have 1.24 million notebooks, none of which overlap between the two lists. Now we can merge the two lists and save the updated csv file about cell data.

In [72]:
df_cell_all = pd.concat([df_cell_reparse, df_cell])
len(df_cell_all.nb_id.unique())

1239355

In [68]:
df_cell_all.to_csv('../data/csv/cell_metadata.csv')

## 3. Data about Cell Content

This will be the more complex parsing of data files. The main things we may want to get for now are imports, headers, and links. 

For the imports, I will need to watch for:
1. Python (import, from)
2. R (library, require)
3. Julia (using, import, importall)

I won't look at other languages as they all had less than 0.1 % of the full sample. 

For headers we can look for # in the markdown as well as the header cells in notebooks written in nbformat 2.0 - 3.0.

For links we may just use a regular expression. However, will this capture relative links? For that we may need to do another regular expression and check if the value returned looks like a url. If not, we may assume it is a relative link.

In [3]:
# Cell metadata
df_cell = pd.read_csv('../data/csv/cell_metadata.csv')
#df_nb.rename(columns = {'Unnamed: 0':'nb_id'}, inplace = True)
print(df_cell.shape)
df_cell.head()

(34690935, 16)


,Unnamed: 0,Unnamed: 0.1,cell_index,cell_type,display_data_keys,error_names,error_values,execute_result_keys,lines_of_code,nb_id,num_display_data,num_error,num_execute_result,num_stream,num_words,workbook_index
0,0,NaN,0,heading,[],[],[],[],NaN,609,0,0,0,0,4.0,0.0
1,1,NaN,1,markdown,[],[],[],[],NaN,609,0,0,0,0,5.0,0.0
2,2,NaN,2,code,[],[],[],[],1.0,609,0,0,0,1,NaN,0.0
3,3,NaN,3,code,[],[],[],[],1.0,609,0,0,0,0,NaN,0.0
4,4,NaN,4,markdown,[],[],[],[],NaN,609,0,0,0,0,13.0,0.0


In [12]:
df_nb_sample = df_nb.sample(1000)

In [116]:
def write_to_log(msg, log_name):
    f = '../logs/%s.txt' % log_name
    log_file = open(f, "a")
    log_file.write(msg + "\n")

def get_cell_content_data(df, tracking_trigger = 10000):
    
    all_cells = []      
    count = 0    
    time1 = time.time()
    
    for index, row in df.iterrows():
        
        # track progress through notebooks
        count += 1
        if count % tracking_trigger == 0:
            print('%s nbs in %s seconds' % (count, time.time() - time1))

        f = '../data/notebooks/nb_%s.ipynb' % row['nb_id']
        with open(f) as nb_file:
            date_string = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            try:
                data = json.load(nb_file)
            except:
                msg = '%s: had trouble loading nb %s' % (date_string, row['nb_id'])
                write_to_log(msg, 'cell_parse_content')
                continue

            if isinstance(data, dict): 
                keys = data.keys()
            else:
                keys = []
                
            # get the language
            nb_language = None
            if 'metadata' in keys:
                if isinstance(data, dict):
                    metadata_keys = data['metadata'].keys()
                else:
                    metadata_keys = []
            else:
                metadata_keys = []
            if 'language_info' in metadata_keys:
                if isinstance(data['metadata']['language_info'], dict):
                    lang_keys = data['metadata']['language_info'].keys()
                else:
                    lang_keys = None
                if 'name' in lang_keys:
                    nb_language = data['metadata']['language_info']['name']
            elif 'language' in metadata_keys:
                nb_language = data['metadata']['language']

            # get data for each cell, nbformat v 4.x
            if 'cells' in keys:
                for i, c in enumerate(data['cells']):
                    cell_data = get_cell_data(c, i, row['nb_id'], nb_language)
                    all_cells.append(cell_data)
            
            # get data for each cell, nbformat v 2.x / 3.x
            elif 'worksheets' in keys:
                for j, w in enumerate(data['worksheets']):
                    if isinstance(w, dict): 
                        worksheet_keys = w.keys()
                    else:
                        keys = []
                    if 'cells' in worksheet_keys:
                        for k, c in enumerate(w['cells']):
                            cell_data = get_cell_data(c, k, row['nb_id'], nb_language, j)
                            all_cells.append(cell_data)
                
    return all_cells
   

def get_cell_data(cell, index, nb_id, nb_language, worksheet_index = None):

    if isinstance(cell, dict): 
        cell_keys = cell.keys()
    else:
        cell_keys = [] 
    
    # get the cell type
    if 'cell_type' in cell_keys:
        cell_type = cell['cell_type']
    else:
        cell_type = None
    
    headings = []
    links = []
    imports = []
    
   # get the imports for python, Julia, and R
    if cell_type == 'code':
        
        lines_of_code = []
        
        if 'source' in cell_keys:
            if isinstance(cell['source'], list):
                lines_of_code = cell['source']
            elif isinstance(cell['source'], str):
                lines_of_code = cell['source'].splitlines()
            
        elif 'input' in cell_keys:
            if isinstance(cell['input'], list):
                lines_of_code = cell['input']
            elif isinstance(cell['input'], str):
                lines_of_code = cell['input'].splitlines()
                
        if nb_language == 'python':
            import_words = ['import', 'from']
        elif nb_language == 'R':
            import_words = ['library', 'require']
        elif nb_language in ['Julia', 'julia']:
            import_words = ['import', 'importall', 'using']
        else:
            import_words = []
            
        # get imports
        for l in lines_of_code:
            words = [x for x in re.split('\(|\)|\s*', l)if x]
            if len(words) >1:
                if words[0] in import_words:                    
                    imports.append(words[1])
    
    
    elif cell_type in ['heading', 'markdown']:
        
        
        # get the lines of markdown
        lines_of_markdown = []
        if 'source' in cell_keys:
            if isinstance(cell['source'], list):
                lines_of_markdown = cell['source']
            elif isinstance(cell['source'], str):
                lines_of_markdown = cell['source'].splitlines()
        
        # track headings in heading cells
        if 'level' in cell_keys:
            if 'source' in cell_keys:
                if len(lines_of_markdown) > 0:
                    headings.append([cell['level'], cell['source']])
        
        for l in lines_of_markdown:
            # get headings in markdown cells
            words = [x for x in re.split('\s*', l)if x]
            if len(words) > 2:
                if words[0] in ['#', '##', '###', '####', '#####', '######']:
                    headings.append([len(words[0]), l.split(words[0])[1]])


            # get links
            urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', l)
            all_links = re.findall('\[([^]]+)]\(\s*(\S*)\)', l)
            for al in all_links:
                if al[:4] != 'http':
                    urls.append(al)
            for u in urls:
                links.append(u)
                
    return [nb_id, nb_language, index, worksheet_index, len(imports), imports, len(headings), headings, len(links), links]
        

In [117]:
all_cell_content = get_cell_content_data(df_nb)

/Users/doug/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


10000 nbs in 89.49553108215332 seconds
20000 nbs in 152.4832980632782 seconds
30000 nbs in 299.51114320755005 seconds
40000 nbs in 362.0098571777344 seconds
50000 nbs in 423.6947419643402 seconds
60000 nbs in 557.3618211746216 seconds
70000 nbs in 615.9819240570068 seconds
80000 nbs in 678.034924030304 seconds
90000 nbs in 874.8168749809265 seconds
100000 nbs in 948.9306139945984 seconds
110000 nbs in 1002.5217590332031 seconds
120000 nbs in 1071.4504733085632 seconds
130000 nbs in 1131.0978531837463 seconds
140000 nbs in 1190.4886672496796 seconds
150000 nbs in 1372.1092112064362 seconds
160000 nbs in 1427.0663442611694 seconds
170000 nbs in 1477.3288660049438 seconds
180000 nbs in 1568.324743270874 seconds
190000 nbs in 1624.2207202911377 seconds
200000 nbs in 1885.0916562080383 seconds
210000 nbs in 1957.902288198471 seconds
220000 nbs in 2029.415874004364 seconds
230000 nbs in 2096.896448135376 seconds
240000 nbs in 2161.0267741680145 seconds
250000 nbs in 2315.6539692878723 second

In [118]:
df_cell_content = pd.DataFrame(all_cell_content)

In [126]:
df_cell_content.columns = ['nb_id', 'nb_language', 'cell_index', 'workbook_index', 
                           'num_imports', 'imports', 'num_headers', 'headers',
                           'num_links', 'links']

In [139]:
df_cell_meta_content = df_cell.merge(df_cell_content, how = 'left', on = ['nb_id', 'cell_index', 'workbook_index'])

In [143]:
del df_cell_meta_content['Unnamed: 0']
del df_cell_meta_content['Unnamed: 0.1']
del df_cell_meta_content['nb_language']

In [144]:
df_cell_meta_content.to_csv('../data/csv/cell_metadata_content.csv')

And now to the analysis! I think

## 4.0 More Data about Cell (code) Content 

After doing some of the later analysis for this project, we decided that we want a little more data about each notebook. Namely the code contents with comments, use of functions, and so on.

In [2]:
# Load cell metadata
df_cell = pd.read_csv('../data/csv/cell_metadata_content.csv')

print(df_cell.shape[0])
df_cell.head()

34690935


,Unnamed: 0,cell_index,cell_type,display_data_keys,error_names,error_values,execute_result_keys,lines_of_code,nb_id,num_display_data,...,num_execute_result,num_stream,num_words,workbook_index,num_imports,imports,num_headers,headers,num_links,links
0,0,0,heading,[],[],[],[],NaN,609,0,...,0,0,4.0,0.0,0,[],1,"[[1, ['NumPy and Matplotlib examples']]]",0,[]
1,1,1,markdown,[],[],[],[],NaN,609,0,...,0,0,5.0,0.0,0,[],0,[],0,[]
2,2,2,code,[],[],[],[],1.0,609,0,...,0,1,NaN,0.0,0,[],0,[],0,[]
3,3,3,code,[],[],[],[],1.0,609,0,...,0,0,NaN,0.0,0,[],0,[],0,[]
4,4,4,markdown,[],[],[],[],NaN,609,0,...,0,0,13.0,0.0,0,[],0,[],0,[]


In [13]:
# get list of nbs to check
nbs_w_cells = df_cell['nb_id'].unique()
nbs_w_cells.shape[0]

df_nbs_cells = df_nb[df_nb['nb_id'].isin(nbs_w_cells)]

In [63]:
def write_to_log(msg, log_name):
    f = '../logs/%s.txt' % log_name
    log_file = open(f, "a")
    log_file.write(msg + "\n")

def get_cell_code_metadata(df, tracking_trigger = 10000):
    
    all_cells = []      
    count = 0    
    time1 = time.time()
    
    for index, row in df.iterrows():
        
        # track progress through notebooks
        count += 1
        if count % tracking_trigger == 0:
            print('%s nbs in %s seconds' % (count, time.time() - time1))

        f = '../data/notebooks/nb_%s.ipynb' % row['nb_id']
        with open(f) as nb_file:
            date_string = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            try:
                data = json.load(nb_file)
            except:
                msg = '%s: had trouble loading nb %s' % (date_string, row['nb_id'])
                write_to_log(msg, 'cell_parse_code_content')
                continue

            if isinstance(data, dict): 
                keys = data.keys()
            else:
                keys = []

            # get the language
            nb_language = None
            if 'metadata' in keys:
                if isinstance(data, dict):
                    metadata_keys = data['metadata'].keys()
                else:
                    metadata_keys = []
            else:
                metadata_keys = []
            if 'language_info' in metadata_keys:
                if isinstance(data['metadata']['language_info'], dict):
                    lang_keys = data['metadata']['language_info'].keys()
                else:
                    lang_keys = None
                if 'name' in lang_keys:
                    nb_language = data['metadata']['language_info']['name']
            elif 'language' in metadata_keys:
                nb_language = data['metadata']['language']
                
            # get data for each cell, nbformat v 4.x
            if 'cells' in keys:
                for i, c in enumerate(data['cells']):
                    cell_data = get_cell_data(c, i, row['nb_id'], nb_language)
                    all_cells.append(cell_data)
            
            # get data for each cell, nbformat v 2.x / 3.x
            elif 'worksheets' in keys:
                for j, w in enumerate(data['worksheets']):
                    if isinstance(w, dict): 
                        worksheet_keys = w.keys()
                    else:
                        keys = []
                    if 'cells' in worksheet_keys:
                        for k, c in enumerate(w['cells']):
                            cell_data = get_cell_data(c, k, row['nb_id'], nb_language, j)
                            all_cells.append(cell_data)
                
    return all_cells

def get_cell_data(cell, index, nb_id, nb_language, worksheet_index = None):

    if isinstance(cell, dict): 
        cell_keys = cell.keys()
    else:
        cell_keys = [] 
    
    # get the cell type
    if 'cell_type' in cell_keys:
        cell_type = cell['cell_type']
    else:
        cell_type = None
    
    functions = []
    classes = []
    comments = 0
    comments_words = 0
    in_multiline = False
    
   # get the imports for python, Julia, and R
    if cell_type == 'code':
        
        # get lines of code
        lines_of_code = []
        if 'source' in cell_keys:
            if isinstance(cell['source'], list):
                lines_of_code = cell['source']
            elif isinstance(cell['source'], str):
                lines_of_code = cell['source'].splitlines()          
        elif 'input' in cell_keys:
            if isinstance(cell['input'], list):
                lines_of_code = cell['input']
            elif isinstance(cell['input'], str):
                lines_of_code = cell['input'].splitlines()
                
        if nb_language == 'python':
            for l in lines_of_code:
                l = l.lstrip()
                parts = l.split()
                if len(parts) >= 2:                
                    if l.startswith('def'):
                        functions.append(parts[1].split('(')[0])
                    elif l.startswith('class'):
                        classes.append(parts[1].split('(')[0])
                
                if in_multiline:
                    comments += 1
                    comments_words += len(parts) - 1
                
                if l.startswith('"""'):
                    if not in_multiline:
                        comments += 1
                        comments_words += len(parts) - 1
                    in_multiline = not in_multiline
                elif l.startswith("'''"):
                    if not in_multiline:
                        comments += 1
                        comments_words += len(parts) - 1
                    in_multiline = not in_multiline
                elif l.startswith('#'):
                    comments += 1
                    comments_words += len(parts) - 1
                
                
    return [nb_id, nb_language, index, worksheet_index, len(functions), functions, len(classes), classes, comments, comments_words]

In [64]:
cell_code_content = get_cell_code_metadata(df_nbs_cells)

10000 nbs in 100.05687093734741 seconds
20000 nbs in 154.06442880630493 seconds
30000 nbs in 294.8419370651245 seconds
40000 nbs in 347.7073030471802 seconds
50000 nbs in 398.4782769680023 seconds
60000 nbs in 515.4168570041656 seconds
70000 nbs in 567.0472660064697 seconds
80000 nbs in 612.7847809791565 seconds
90000 nbs in 801.9396131038666 seconds
100000 nbs in 848.1235349178314 seconds
110000 nbs in 892.9966039657593 seconds
120000 nbs in 945.7173871994019 seconds
130000 nbs in 993.585531949997 seconds
140000 nbs in 1093.6600029468536 seconds
150000 nbs in 1260.6267700195312 seconds
160000 nbs in 1302.7648758888245 seconds
170000 nbs in 1355.7270030975342 seconds
180000 nbs in 1408.9285321235657 seconds
190000 nbs in 1456.095136165619 seconds
200000 nbs in 1740.700397014618 seconds
210000 nbs in 1788.9886770248413 seconds
220000 nbs in 1834.3918809890747 seconds
230000 nbs in 1896.7951159477234 seconds
240000 nbs in 2057.247360944748 seconds
250000 nbs in 2106.0779650211334 seconds

In [65]:
df_code = pd.DataFrame(cell_code_content)

In [74]:
df_code.columns = ['nb_id',
                    'language',
                    'cell_index',
                    'workbook_index',
                    'num_functions',
                    'name_functions',
                    'num_classes',
                    'name_classes',
                    'num_comment_lines',
                    'num_comment_words']
df_code.head()

,nb_id,language,cell_index,workbook_index,num_functions,name_functions,num_classes,name_classes,num_comment_lines,num_comment_words
0,585,python,0,NaN,0,[],0,[],0,0
1,585,python,1,NaN,0,[],0,[],0,0
2,585,python,2,NaN,0,[],0,[],2,38
3,585,python,3,NaN,0,[],0,[],0,0
4,585,python,4,NaN,7,"[wavwrite, stft, spectrogram, mel2freq, freq2m...",0,[],141,858


In [69]:
df_code.to_csv('../data/csv/cell_metadata_code.csv')

And that's a wrap. Now we should have almost all the data we need to do our [core analysis of the notebooks](7_notebook_profiling.ipynb)